### 사용자 정의 모델 클래스
- 부모 클래스: nn.Module
- 필수 오버라이딩
	* __init__(): 모델 층 구성, 설계
	* forward(): 순방향 학습 진행 코드 구현
- 동적 모델
	* container 모듈 중 nn.ModuleList() 사용해서 동적으로 Layer 추가 
		- forward 기능 미제공
		- layer 인스턴스 요소 사이 연관성 X
		- layer 인스턴스 요소 => 인덱싱 접근

In [23]:
import torch								# 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn						# 인공신경망 관련 모듈
import torch.nn.functional as F 			# 손실, 거래 등 함수 관련 모듈
import torch.optim as optimizer 			# 최적화 기법 관련 모듈
from torchinfo import summary				# 모델 구조 및 정보 관련 모듈

from torchmetrics.regression import *		# 회귀 성능 지표 모듈
from torchmetrics.classification import *	# 분류 성능 지표 모듈

In [24]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 고정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- 동적 모델 설계 <hr>
	* 목표: 은닉층의 개수가 동적인 모델
	* 조건:
		* 입력층 & 출력층 개수 동적 => 입력층의 입력값, 출력층의 출력값
		* 은닉층의 개수 동적 + 퍼셉트론 개수 고정		=> 은닉층의 개수, 퍼셉트론 수 

In [25]:
# ---------------------------------------------------------
# 모델이름: DynamicModel
# 부모클래스: nn.module
# 매개변수: in_in, out_out, h_inout, h_cnt
# ---------------------------------------------------------

class DynamicModel(nn.Module):

	# 모델 구조 설계 함수 즉, 생성자 메서드
	def __init__(self, in_in, out_out, h_inout, h_cnt ):
        
		super().__init__()

		self.in_layer = nn.Linear(in_in, h_inout)
		# nn.ModuleList => 
		self.h_layers = nn.ModuleList( [ nn.Linear(h_inout, h_inout) for _ in range(h_cnt) ] )
		self.out_layer = nn.Linear(h_inout, out_out)

	# 학습 진행 콜백 메서드
	# 함수 이름 무조건 'forward'
	def forward(self, x):

		# 입력층
		y = self.in_layer(x)			# y=x1w1+x2w2+x3w3+ b
		y = F.relu(y)					# 0 <= y

		# y = F.relu( self.in_layer(x))


		# 은닉층
		for linear in self.h_layers:
			y = linear(y)
			y = F.relu(y)
			# y = F.relu( self.in_layer(x))
			
		# 출력층
		return self.out_layer(y)

In [26]:
# 모델 인스턴스 생성
m1 = DynamicModel(3,2, 5,10)

# 모델 구조 확인
print(m1)

DynamicModel(
  (in_layer): Linear(in_features=3, out_features=5, bias=True)
  (h_layers): ModuleList(
    (0-9): 10 x Linear(in_features=5, out_features=5, bias=True)
  )
  (out_layer): Linear(in_features=5, out_features=2, bias=True)
)


In [27]:
# 모델 파라미터 확인
for name, param in m1.named_parameters():
    print(name, param.shape)

in_layer.weight torch.Size([5, 3])
in_layer.bias torch.Size([5])
h_layers.0.weight torch.Size([5, 5])
h_layers.0.bias torch.Size([5])
h_layers.1.weight torch.Size([5, 5])
h_layers.1.bias torch.Size([5])
h_layers.2.weight torch.Size([5, 5])
h_layers.2.bias torch.Size([5])
h_layers.3.weight torch.Size([5, 5])
h_layers.3.bias torch.Size([5])
h_layers.4.weight torch.Size([5, 5])
h_layers.4.bias torch.Size([5])
h_layers.5.weight torch.Size([5, 5])
h_layers.5.bias torch.Size([5])
h_layers.6.weight torch.Size([5, 5])
h_layers.6.bias torch.Size([5])
h_layers.7.weight torch.Size([5, 5])
h_layers.7.bias torch.Size([5])
h_layers.8.weight torch.Size([5, 5])
h_layers.8.bias torch.Size([5])
h_layers.9.weight torch.Size([5, 5])
h_layers.9.bias torch.Size([5])
out_layer.weight torch.Size([2, 5])
out_layer.bias torch.Size([2])


* 학습 진행

In [28]:
## 임시 데이터 생성
dataTS = torch.FloatTensor([ [1,3,5], [2,4,6], [3,5,7], [4,6,8]]) 	# 2D  4행 3열(4,3)
targetTS = torch.FloatTensor( [ [7,9], [8,10], [9,11], [10,12] ])	# 2D  4행 2열(4,2)

In [29]:
# 모델 학습
pre_y = m1(dataTS)

print(pre_y, pre_y.shape, sep='\n')

tensor([[0.1237, 0.0215],
        [0.1237, 0.0215],
        [0.1237, 0.0215],
        [0.1237, 0.0215]], grad_fn=<AddmmBackward0>)
torch.Size([4, 2])


In [ ]:
# 입력층의 개수가 동적인 모델

In [ ]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module):
    
    # 인스턴스/객체 생성 시 자동호출 메서드
    def __init__(self,in_features, in_out, hidden_cnt, **h_out):
    
    # 부모 클래스 생성
        super().__init__()
    
    # 자식클래스 인스턴스 속성 설정
    
		if hidden_cnt == 0:	
			self.input_layer = nn.Linear(in_features, in_out)	
			self.output_layer = nn.Linear(in_out, 1)
        
		elif hidden_cnt >0:
			self.input_layer = nn.Linear(in_features, in_out)
			self.hidden_yaer = nn.Linear(in_out,h_out)
			self.output_layer = nn.Linear(h_out, 1)

		else:
			print(f'은닉층 반복횟수는 0또는 양수')
    
	# Callback func
	# 		=> 순반향/전방향 학습 시, 자동호출되는 메서드
	#		=> 시스템에서 호출
	# 		=> 전달인자: Train DS
    def forward(self, x, hidden_cnt):
        print('calling forward()')
        
        if hidden_cnt == 0:
             y = self.input_layer(x)
			 
		
        y = self.input_layer(x)		# 1 Perceptron -> y = x1w1+x2w2+x3w3+x4w4 + b
									#											--> 20개
        y = F.relu(y)				# 0<= y 	--> 죽은 ReLU 발생 --> leakyReLU로 해결
		
        y = self.hidden_layer(y)	# 1 Perceptron -> y = x1w1+ ... + x20w20 + b
									# 											--> 100개
        y = F.relu(y)
		
        return self.output_layer(y)	# 1 Perceptron -> y = x1w1+ ... + x100w100 + b

In [18]:
# 모델 인스턴스 생성
m1 = MyModel(4)

In [6]:
# for m in m1.parameters(): print(m)

In [ ]:
# for m in m1.named_parameters(): print(m)

In [19]:
# 학습 진행 ==> 모델인스턴스명(데이터)

# 임의의 데이터
dataTS = torch.FloatTensor( [[1,3,5,7], [2,4,6,8]])
targetTS = torch.FloatTensor( [[4],[5]] )

# 학습
pre_y = m1(dataTS)

print(pre_y)

calling forward()
tensor([[-0.1395],
        [-0.1858]], grad_fn=<AddmmBackward0>)
